## Setup

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

## TensorFlow Core Walkthrough

### Graph

In [2]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


### TensorBoard

In [3]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

### Session

In [4]:
sess = tf.Session()
print(sess.run(total))

7.0


In [5]:
print(sess.run({'ab': (a, b), 'total': total}))

{'ab': (3.0, 4.0), 'total': 7.0}


In [6]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.93953085 0.14691675 0.688282  ]
[0.58099544 0.9487721  0.550362  ]
(array([1.2370502, 1.8748823, 1.613271 ], dtype=float32), array([2.23705  , 2.8748822, 2.613271 ], dtype=float32))


### Feeding

In [7]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [8]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y:[2, 4]}))

7.5
[3. 7.]


## Datasets

In [9]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

In [10]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


In [11]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        break

[-0.680384  -1.3080088 -0.5602123]
[ 0.74375445 -0.7269249   0.8641697 ]
[-0.01520354 -0.01904213  0.6500065 ]
[-0.75541   -0.3295451 -0.5917274]
[-0.6279358  -0.9601611  -0.35794324]
[ 0.8481068   0.19351372 -0.9667384 ]
[0.22194256 1.2835034  0.9430937 ]
[-0.50041634 -0.856591   -0.97646356]
[ 0.24655584 -1.8106551   0.46335483]
[-0.2345854  -2.074828    0.03459405]


## Layers

### Creating Layers

In [12]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

### Initializing Layers

In [13]:
init = tf.global_variables_initializer()
sess.run(init)

### Executing Layers

In [14]:
print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[-0.6942595]
 [-1.4782119]]


### Layer Function shortcuts

In [15]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[-0.5516398 ]
 [ 0.07637787]]


## Feature columns

In [16]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

In [17]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

In [18]:
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


## Training

### Define the data

In [19]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

### Define the model

In [20]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

In [21]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[-1.7237554]
 [-3.4475107]
 [-5.171266 ]
 [-6.8950214]]


### Loss

In [22]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

8.547441


### Training

In [23]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [24]:
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

8.547441
5.9537735
4.1539454
2.9049463
2.038158
1.4365766
1.0190188
0.72915155
0.5278868
0.38810235
0.2909785
0.22345656
0.1764757
0.14374854
0.12091254
0.104940556
0.09373216
0.085829824
0.08022224
0.07620772
0.07329928
0.07115912
0.06955273
0.06831745
0.0673404
0.06654325
0.06587163
0.065287836
0.0647657
0.06428702
0.0638392
0.063413486
0.06300379
0.06260588
0.062216878
0.061834697
0.06145789
0.061085556
0.060716897
0.060351513
0.05998904
0.059629254
0.059271943
0.058917
0.058564365
0.058213983
0.05786574
0.05751963
0.05717565
0.05683376
0.056493893
0.056156114
0.05582034
0.0554866
0.055154838
0.05482506
0.054497257
0.054171443
0.053847533
0.053525582
0.053205572
0.052887477
0.052571256
0.05225693
0.051944513
0.05163395
0.051325236
0.051018354
0.05071334
0.05041012
0.05010873
0.049809143
0.049511336
0.049215324
0.048921075
0.04862857
0.048337813
0.04804881
0.047761545
0.047475975
0.04719215
0.04690998
0.046629515
0.046350714
0.04607361
0.045798134
0.045524307
0.045252115
0.044981584


### Complete program

In [25]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

print(sess.run(y_pred))

2.4525962
1.7363069
1.2390822
0.8938631
0.65411925
0.4875635
0.37179255
0.29126143
0.2351836
0.19607458
0.16874099
0.14957933
0.13608924
0.12653558
0.119714566
0.11479081
0.11118464
0.10849386
0.10643937
0.104827516
0.103523895
0.10243532
0.101496965
0.10066402
0.09990523
0.09919909
0.09853044
0.09788892
0.09726728
0.096660465
0.09606507
0.09547855
0.09489927
0.09432602
0.09375801
0.09319463
0.0926355
0.09208026
0.09152881
0.090980925
0.0904365
0.08989547
0.0893578
0.08882337
0.088292204
0.08776425
0.08723947
0.08671784
0.086199336
0.08568395
0.08517165
0.08466241
0.08415623
0.08365304
0.083152905
0.082655765
0.08216156
0.08167033
0.08118205
0.08069664
0.0802142
0.079734586
0.0792579
0.07878402
0.07831296
0.077844754
0.07737932
0.07691667
0.07645683
0.075999685
0.07554532
0.07509365
0.07464466
0.074198365
0.07375477
0.0733138
0.07287545
0.07243976
0.072006635
0.07157612
0.07114818
0.070722796
0.07029995
0.069879636
0.06946185
0.069046535
0.06863372
0.06822336
0.06781547
0.06741
0.06700